In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
%cd gdrive/My\ Drive/Colab\ Notebooks/'word_generator'

/content/gdrive/My Drive/Colab Notebooks/word_generator


In [0]:
#%cd  word_generator/

/content/gdrive/My Drive/Colab Notebooks/word_generator


In [0]:
#!wget http://www.gutenberg.org/cache/epub/1497/pg1497.txt

--2019-11-05 20:10:42--  http://www.gutenberg.org/cache/epub/1497/pg1497.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1239081 (1.2M) [text/plain]
Saving to: ‘pg1497.txt’

pg1497.txt          100%[===================>]   1.18M   647KB/s    in 1.9s    

2019-11-05 20:10:45 (647 KB/s) - ‘pg1497.txt’ saved [1239081/1239081]



In [0]:
#!mv pg1497.txt republic.txt

Text processing

In [0]:
import string

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# turn a doc into clean tokens
def clean_doc(doc):
	# replace '--' with a space ' '
	doc = doc.replace('--', ' ')
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# make lower case
	tokens = [word.lower() for word in tokens]
	return tokens

# save tokens to file, one dialog per line
def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()
 

In [0]:
# load document
in_filename = 'republic_clean.txt'
doc = load_doc(in_filename)
print(doc[:200])

# clean document
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences.append(line)
print('Total Sequences: %d' % len(sequences))

# save sequences to file
out_filename = 'republic_sequences.txt'
save_doc(sequences, out_filename)

FileNotFoundError: ignored

In [0]:
sequences = load_doc('republic_sequences.txt')

In [0]:
sequences[0:300]

'i the republic opens with a truly greek scene a festival in honour of the goddess bendis which is held in the piraeus to this is added the promise of an equestrian torchrace in the evening the whole work is supposed to be recited by socrates on the day after the\nthe republic opens with a truly greek'

In [0]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

# load
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

# define model
model = Sequential()
# in : (batch, input_lengh) , [12,3,2,3,4,5,12,..., input_lentgh]
model.add(Embedding(vocab_size, 50, input_length=seq_length))
#out (batch, input_length, vector_size) = (batch, 50,50)
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=100)

# save the model to file
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

Using TensorFlow backend.





Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            513550    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 10271)             1037371   
Total params: 1,701,821
Trainable params: 1,701,821
Non-trainable params: 0
_________________________________________________________________
None


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epo

##|Text generation

In [0]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [0]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	# import pdb; pdb.set_trace()
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# yhat2= model.predict(encoded) 
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)

In [0]:


# load cleaned text sequences
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1

# load the model
model = load_model('model.h5')

# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))

# select a seed text
seed_text = lines[30]#lines[randint(0,len(lines))]
print(seed_text + '\n')

# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

Text gen specific words

In [0]:
def words_to_index(tokenizer,words):
  words_indices = []
  for word in words:
    words_indices.append(tokenizer.word_index[word])
  return words_indices

In [0]:
import numpy as np
import random
# generate a sequence from a language model

def generate_seq_from_seed_words(model, tokenizer, seq_length, seed_words, n_words=50,alpha=0.0000001):
	"""
	Generate text using (mnemonic) that include the words from seed_words


	Arguments:
	seed_words -- the set of words to generat the text. For kanji would be the radicals.
	They are encoded as integers. 
	alpha -- minimal probability for including the word
	"""
	result = list()
	index_first_word = random.randint(0,len(seed_words)-1)
	in_text = seed_words[index_first_word]
	print('Starting word:', in_text)
	del seed_words[index_first_word]
	seed_words_indices = words_to_index(tokenizer,seed_words)

	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# import pdb; pdb.set_trace()
		# predict probabilities for each word
		y_probs = model.predict(encoded)[0]
		seed_words_probs = y_probs[seed_words_indices]
		# print(seed_words_probs)
		yhat = np.argmax(y_probs)
		# print(y_probs[yhat])
		out_word = ''
		search_on_all = True
		for i,prob in enumerate(seed_words_probs):
			if prob >= alpha:
				out_word = seed_words[i]
				del seed_words[i]
				seed_words_indices = words_to_index(tokenizer,seed_words)
				search_on_all = False

		#dont search if out_words is from seed_words		
		if search_on_all:
			# map predicted word index to word
			for word, index in tokenizer.word_index.items():
				if index == yhat:
					out_word = word
					break

		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)


In [0]:
tokenizer.word_index.items()

dict_items([('the', 1), ('of', 2), ('and', 3), ('to', 4), ('is', 5), ('in', 6), ('a', 7), ('he', 8), ('that', 9), ('be', 10), ('which', 11), ('not', 12), ('or', 13), ('are', 14), ('i', 15), ('they', 16), ('as', 17), ('will', 18), ('but', 19), ('have', 20), ('you', 21), ('we', 22), ('by', 23), ('them', 24), ('his', 25), ('for', 26), ('said', 27), ('with', 28), ('their', 29), ('who', 30), ('this', 31), ('one', 32), ('there', 33), ('all', 34), ('at', 35), ('has', 36), ('what', 37), ('may', 38), ('if', 39), ('from', 40), ('when', 41), ('then', 42), ('other', 43), ('no', 44), ('him', 45), ('state', 46), ('good', 47), ('any', 48), ('an', 49), ('was', 50), ('only', 51), ('would', 52), ('do', 53), ('man', 54), ('true', 55), ('more', 56), ('on', 57), ('our', 58), ('than', 59), ('like', 60), ('must', 61), ('yes', 62), ('were', 63), ('can', 64), ('should', 65), ('been', 66), ('us', 67), ('men', 68), ('such', 69), ('say', 70), ('into', 71), ('so', 72), ('also', 73), ('it', 74), ('own', 75), ('now'

 ('renowned', 3785)
 'monarchia', 8139
 ('victory', 2106)

In [0]:
words = ['renowned', 'monarchia','victory']
words_to_index(tokenizer,words)

[3785, 8139, 2106]

In [0]:
# load cleaned text sequences
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1

# load the model
model = load_model('model.h5')

# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))


In [0]:

# select a seed text
seed_words =  ['renowned', 'monarchia','victory']
print(seed_words,'\n')

# generate new text
generated = generate_seq_from_seed_words(model, tokenizer, seq_length, seed_words, 50)
print(generated)

['renowned', 'monarchia', 'victory'] 

Starting word: renowned
but assuredly must they look satisfied and opinion and about the other victory and gymnastic which await play her hour or some other sinners which er from men and women or women or women or to ideas unmanliness he learned besides the soul of gentleness and a profession of diseases


## Saving word emmbedings

In [0]:
layer_emb = model.layers[0]
weights = layer_emb.get_weights()[0]
print(weights.shape)

(10271, 50)


In [0]:
import io

vector_out = io.open('vects.tsv', 'w', encoding = 'utf-8')
words_out = io.open('words.tsv', 'w', encoding = 'utf-8')


for word,i in tokenizer.word_index.items():
  vec = weights[i-1]
  words_out.write(word + '\n')
  vector_out.write(  '\t'.join( [str(vec[j]) for j in range(vec.shape[0])] ) + '\n')


vector_out.close()
words_out.close()

## Generate sequence from sampling

In [0]:
def sample(preds, temperature=1.0):
  """
  The higher the temperature the more softer probabilities. Which results in more diversity.
  The lower the temperature the higher the probabilities, and the choos more
  strongly values with high proability

  """

  if temperature <= 0:
    return np.argmax(preds)
  preds = np.asarray(preds).astype('float64')
  #gives RuntimeWarning: divide by zero encountered in log, probably, is too near
  #to zero.
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  #Draw the index of preds, using probabilities from preds. 
  # if   [0.1,0.8,0.1] then 1 is drawn 80% of the time
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [0]:
sample([0.1,0.8,0.1],temperature=100)

2

In [0]:
# generate a sequence from a language model
def generate_seq_with_sampling(model, tokenizer, seq_length, seed_text, n_words):
  result = list()
  in_text = seed_text
  # generate a fixed number of words
  # import pdb; pdb.set_trace()
  for _ in range(n_words):
    # encode the text as integer
    encoded = tokenizer.texts_to_sequences([in_text])[0]
    # truncate sequences to a fixed length
    encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
    # predict probabilities for each word

    probs = model.predict(encoded, verbose=0)[0]
    yhat = sample(probs)
    # map predicted word index to word
    out_word = ''
    for word, index in tokenizer.word_index.items():
      if index == yhat:
        out_word = word
        break
      # append to input
    in_text += ' ' + out_word
    result.append(out_word)
  return ' '.join(result)

In [0]:
# load cleaned text sequences
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1

# load the model
model = load_model('model.h5')

# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))

# select a seed text
seed_text = lines[30]#lines[randint(0,len(lines))]
print(seed_text + '\n')

# generate new text
generated = generate_seq_with_sampling(model, tokenizer, seq_length, seed_text, 50)
print(generated)

an equestrian torchrace in the evening the whole work is supposed to be recited by socrates on the day after the festival to a small party consisting of critias timaeus hermocrates and another this we learn from the first words of the timaeus when the rhetorical advantage of reciting the dialogue



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in log
  


he prefers statesman was brought again at last as an oystershell which led all harmonized with a physical pleasures either with an effort and pol by plato met and surely he said i am sure of the gentle names and are differently clothed or meanness certainly not suppose that they


In [0]:
# select a seed text
seed_text = 'monarchia'#lines[randint(0,len(lines))]
print(seed_text + '\n')

# generate new text
generated = generate_seq_with_sampling(model, tokenizer, seq_length, seed_text, 50)
print(generated)

In [0]:
# select a seed text
seed_words =  ['renowned', 'monarchia','victory']
print(seed_words,'\n')

In [0]:
word_mapo = tokenizer.word_index

In [0]:
word_mapo['monarchia']

8139

In [0]:
def cosine_similarity(u, v):
    """
    Cosine similarity reflects the degree of similariy between u and v
        
    Arguments:
        u -- a word vector of shape (n,)          
        v -- a word vector of shape (n,)

    Returns:
        cosine_similarity -- the cosine similarity between u and v defined by the formula above.
    """
    
    distance = 0.0
    
    ### START CODE HERE ###
    # Compute the dot product between u and v (≈1 line)
    dot = np.dot(u,v)
    # Compute the L2 norm of u (≈1 line)
    # import pdb; pdb.set_trace()
    norm_u = np.sqrt(np.dot(u,u))
    
    # Compute the L2 norm of v (≈1 line)
    norm_v = np.sqrt(np.dot(v,v))
    # Compute the cosine similarity defined by formula (1) (≈1 line)
    cosine_similarity = dot / (norm_u * norm_v )
    ### END CODE HERE ###
    
    return cosine_similarity

In [0]:
def change_words_in_sequence(sequence,seed_words,word_to_vec_map):
  """
  Change the nearest words  to seed_words in the sequence.
  Args:
  sequence -- a sequence of text 'the big black cat...' 
  seed_worods --  a list of words to sustitute = ['dog', 'white' ]

  returns:
  new text sequence -- eg 'the big white dog...'
  """
  list_sequence = sequence.split(' ')

  for i,word in enumerate(seed_words):
    for j, seq_word in enumerate(list_sequence(' ')):
      #Calculate distance(word,seq_word)







In [0]:
change_words_in_sequence('the big black cat',['dog','white'],tokenizer.word_index)

0 the
1 big
2 black
3 cat
